In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import numpy as np
import matplotlib.pyplot as plt

# Base Model

In [ ]:
train_ds_base = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/train',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

val_ds_base = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/val',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

test_ds_base = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/test',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

model_base = models.Sequential([
    layers.Input(shape=(512, 512, 3)),
    layers.Rescaling(1./255),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_base.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall')
    ]
)

history = model_base.fit(
    train_ds_base,
    validation_data=val_ds_base,
    epochs=30
)

acc_base = history.history['accuracy']
val_acc_base = history.history['val_accuracy']

precision_base = history.history['precision']
val_precision_base = history.history['val_precision']

recall_base = history.history['recall']
val_recall_base = history.history['val_recall']

loss_base = history.history['loss']
val_loss_base = history.history['val_loss']

epochs_range_base = range(len(acc_base))

plt.figure(figsize=(24, 6))

# Accuracy
plt.subplot(1, 4, 1)
plt.plot(epochs_range_base, acc_base, label='Train Accuracy')
plt.plot(epochs_range_base, val_acc_base, label='Val Accuracy')
plt.title('Accuracy vs. Epoch Base')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Precision
plt.subplot(1, 4, 2)
plt.plot(epochs_range_base, precision_base, label='Train Precision')
plt.plot(epochs_range_base, val_precision_base, label='Val Precision')
plt.title('Precision vs. Epoch Base')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# Recall
plt.subplot(1, 4, 3)
plt.plot(epochs_range_base, recall_base, label='Train Recall')
plt.plot(epochs_range_base, val_recall_base, label='Val Recall')
plt.title('Recall vs. Epoch Base')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

# Loss
plt.subplot(1, 4, 4)
plt.plot(epochs_range_base, loss_base, label='Train Loss')
plt.plot(epochs_range_base, val_loss_base, label='Val Loss')
plt.title('Loss vs. Epoch Base')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model on the test dataset
test_loss_base, test_acc_base, test_precision_base, test_recall_base = model_base.evaluate(test_ds_base)

# Print out the results
print(f"Test Accuracy: {test_acc_base:.3f}")
print(f"Test Precision: {test_precision_base:.3f}")
print(f"Test Recall: {test_recall_base:.3f}")
print(f"Test Loss: {test_loss_base:.3f}")

# Batch Size Testing

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import numpy as np
import matplotlib.pyplot as plt

batch_sizes = [8, 16, 32, 64, 128, 256]

# Initialize lists to store metrics for train, val, and test datasets
train_accuracies = []
val_accuracies = []
test_accuracies = []

train_precisions = []
val_precisions = []
test_precisions = []

train_recalls = []
val_recalls = []
test_recalls = []

train_losses = []
val_losses = []
test_losses = []

for current_batch_size in batch_sizes:
    print(f"Training with batch size {current_batch_size}")
    AUTOTUNE = tf.data.AUTOTUNE

    #  Load datasets with the current batch size
    train_ds = tf.keras.utils.image_dataset_from_directory(
        'processed_dataset/train',
        image_size=(512, 512),
        batch_size=current_batch_size,
        label_mode='binary'
    ).cache().prefetch(buffer_size=AUTOTUNE)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        'processed_dataset/val',
        image_size=(512, 512),
        batch_size=current_batch_size,
        label_mode='binary'
    ).cache().prefetch(buffer_size=AUTOTUNE)

    test_ds = tf.keras.utils.image_dataset_from_directory(
        'processed_dataset/test',
        image_size=(512, 512),
        batch_size=current_batch_size,
        label_mode='binary'
    ).cache().prefetch(buffer_size=AUTOTUNE)

    
    # Build and compile the model
    model = models.Sequential([
        layers.Input(shape=(512, 512, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )
    
    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=30
    )
    
    # Evaluate the model
    test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds)
    
    # Print test results
    print(f"Batch Size {current_batch_size}:")
    print(f"Test Accuracy: {test_acc:.3f}")
    print(f"Test Precision: {test_precision:.3f}")
    print(f"Test Recall: {test_recall:.3f}")
    print(f"Test Loss: {test_loss:.3f}")
    
    # Append the test results to the lists for plotting
    test_accuracies.append(test_acc)
    test_precisions.append(test_precision)
    test_recalls.append(test_recall)
    test_losses.append(test_loss)
    
    # Plot results
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    precision = history.history['precision']
    val_precision = history.history['val_precision']
    recall = history.history['recall']
    val_recall = history.history['val_recall']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    train_accuracies.append(acc[-1])
    val_accuracies.append(val_acc[-1])

    train_precisions.append(precision[-1])
    val_precisions.append(val_precision[-1])

    train_recalls.append(recall[-1])
    val_recalls.append(val_recall[-1])

    train_losses.append(loss[-1])
    val_losses.append(val_loss[-1])
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(24, 6))
    
    plt.subplot(1, 4, 1)
    plt.plot(epochs_range, acc, label=f'Train Accuracy {current_batch_size}')
    plt.plot(epochs_range, val_acc, label=f'Val Accuracy {current_batch_size}')
    plt.title(f'Accuracy vs. Epoch Batch {current_batch_size}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 4, 2)
    plt.plot(epochs_range, precision, label=f'Train Precision {current_batch_size}')
    plt.plot(epochs_range, val_precision, label=f'Val Precision {current_batch_size}')
    plt.title(f'Precision vs. Epoch Batch {current_batch_size}')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()
    
    plt.subplot(1, 4, 3)
    plt.plot(epochs_range, recall, label=f'Train Recall {current_batch_size}')
    plt.plot(epochs_range, val_recall, label=f'Val Recall {current_batch_size}')
    plt.title(f'Recall vs. Epoch Batch {current_batch_size}')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    
    plt.subplot(1, 4, 4)
    plt.plot(epochs_range, loss, label=f'Train Loss {current_batch_size}')
    plt.plot(epochs_range, val_loss, label=f'Val Loss {current_batch_size}')
    plt.title(f'Loss vs. Epoch Batch {current_batch_size}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Convert batch sizes to log2 scale
log_batch_sizes = np.log2(batch_sizes)

# Plot the results for train, validation, and test metrics vs batch size on a log2 scale
plt.figure(figsize=(24, 6))

# Accuracy vs Batch Size (log2 scale)
plt.subplot(1, 4, 1)
plt.plot(log_batch_sizes, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(log_batch_sizes, val_accuracies, label='Validation Accuracy', marker='o')
plt.plot(log_batch_sizes, test_accuracies, label='Test Accuracy', marker='o')
plt.title('Accuracy vs. Batch Size (log2 scale)')
plt.xlabel('Log2(Batch Size)')
plt.ylabel('Accuracy')
plt.legend()

# Precision vs Batch Size (log2 scale)
plt.subplot(1, 4, 2)
plt.plot(log_batch_sizes, train_precisions, label='Train Precision', marker='o')
plt.plot(log_batch_sizes, val_precisions, label='Validation Precision', marker='o')
plt.plot(log_batch_sizes, test_precisions, label='Test Precision', marker='o')
plt.title('Precision vs. Batch Size (log2 scale)')
plt.xlabel('Log2(Batch Size)')
plt.ylabel('Precision')
plt.legend()

# Recall vs Batch Size (log2 scale)
plt.subplot(1, 4, 3)
plt.plot(log_batch_sizes, train_recalls, label='Train Recall', marker='o')
plt.plot(log_batch_sizes, val_recalls, label='Validation Recall', marker='o')
plt.plot(log_batch_sizes, test_recalls, label='Test Recall', marker='o')
plt.title('Recall vs. Batch Size (log2 scale)')
plt.xlabel('Log2(Batch Size)')
plt.ylabel('Recall')
plt.legend()

# Loss vs Batch Size (log2 scale)
plt.subplot(1, 4, 4)
plt.plot(log_batch_sizes, train_losses, label='Train Loss', marker='o')
plt.plot(log_batch_sizes, val_losses, label='Validation Loss', marker='o')
plt.plot(log_batch_sizes, test_losses, label='Test Loss', marker='o')
plt.title('Loss vs. Batch Size (log2 scale)')
plt.xlabel('Log2(Batch Size)')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Fully Connected Layers Testing

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import numpy as np
import matplotlib.pyplot as plt

dense_layer_counts = [1, 2, 3, 4]
batch_size = 64  # Fixed

# Initialize lists to store metrics
train_accuracies, val_accuracies, test_accuracies = [], [], []
train_precisions, val_precisions, test_precisions = [], [], []
train_recalls, val_recalls, test_recalls = [], [], []
train_losses, val_losses, test_losses = [], [], []

AUTOTUNE = tf.data.AUTOTUNE

# Load datasets once since batch size is constant
train_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/train',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/val',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/test',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

for dense_layers in dense_layer_counts:
    print(f"\nTraining with {dense_layers} Dense Layers")
    
    # Build model
    model = models.Sequential([
        layers.Input(shape=(512, 512, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten()
    ])
    
    # Add variable number of Dense layers
    for _ in range(dense_layers):
        model.add(layers.Dense(128, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )
    
    # Train model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=30
    )
    
    # Evaluate model
    test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds)
    
    print(f"Dense Layers {dense_layers}:")
    print(f"Test Accuracy: {test_acc:.3f}")
    print(f"Test Precision: {test_precision:.3f}")
    print(f"Test Recall: {test_recall:.3f}")
    print(f"Test Loss: {test_loss:.3f}")
    
    # History metrics
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    precision = history.history['precision']
    val_precision = history.history['val_precision']
    recall = history.history['recall']
    val_recall = history.history['val_recall']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Store final epoch values
    train_accuracies.append(acc[-1])
    val_accuracies.append(val_acc[-1])
    test_accuracies.append(test_acc)

    train_precisions.append(precision[-1])
    val_precisions.append(val_precision[-1])
    test_precisions.append(test_precision)

    train_recalls.append(recall[-1])
    val_recalls.append(val_recall[-1])
    test_recalls.append(test_recall)

    train_losses.append(loss[-1])
    val_losses.append(val_loss[-1])
    test_losses.append(test_loss)
    
    # Plot metrics per epoch
    epochs_range = range(len(acc))
    plt.figure(figsize=(24, 6))
    
    plt.subplot(1, 4, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title(f'Accuracy vs. Epochs - {dense_layers} Dense Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 4, 2)
    plt.plot(epochs_range, precision, label='Train Precision')
    plt.plot(epochs_range, val_precision, label='Val Precision')
    plt.title(f'Precision vs. Epochs - {dense_layers} Dense Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()
    
    plt.subplot(1, 4, 3)
    plt.plot(epochs_range, recall, label='Train Recall')
    plt.plot(epochs_range, val_recall, label='Val Recall')
    plt.title(f'Recall vs. Epochs - {dense_layers} Dense Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    
    plt.subplot(1, 4, 4)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title(f'Loss vs. Epochs - {dense_layers} Dense Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot final metrics vs number of dense layers
plt.figure(figsize=(24, 6))

plt.subplot(1, 4, 1)
plt.plot(dense_layer_counts, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(dense_layer_counts, val_accuracies, label='Val Accuracy', marker='o')
plt.plot(dense_layer_counts, test_accuracies, label='Test Accuracy', marker='o')
plt.title('Accuracy vs. Dense Layers')
plt.xlabel('Number of Dense Layers')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 4, 2)
plt.plot(dense_layer_counts, train_precisions, label='Train Precision', marker='o')
plt.plot(dense_layer_counts, val_precisions, label='Val Precision', marker='o')
plt.plot(dense_layer_counts, test_precisions, label='Test Precision', marker='o')
plt.title('Precision vs. Dense Layers')
plt.xlabel('Number of Dense Layers')
plt.ylabel('Precision')
plt.legend()

plt.subplot(1, 4, 3)
plt.plot(dense_layer_counts, train_recalls, label='Train Recall', marker='o')
plt.plot(dense_layer_counts, val_recalls, label='Val Recall', marker='o')
plt.plot(dense_layer_counts, test_recalls, label='Test Recall', marker='o')
plt.title('Recall vs. Dense Layers')
plt.xlabel('Number of Dense Layers')
plt.ylabel('Recall')
plt.legend()

plt.subplot(1, 4, 4)
plt.plot(dense_layer_counts, train_losses, label='Train Loss', marker='o')
plt.plot(dense_layer_counts, val_losses, label='Val Loss', marker='o')
plt.plot(dense_layer_counts, test_losses, label='Test Loss', marker='o')
plt.title('Loss vs. Dense Layers')
plt.xlabel('Number of Dense Layers')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()